In [2]:
cd ..

/home/amiyaguchi/cs224w/wikipedia-retention


In [3]:
import pandas as pd

df = pd.read_csv("data/processed/rolx-v", header=None, sep=' ')

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,6425.0,1252030.0,5437202.0,7934837.0,1234.994141,1.097501e+09,170817.281250,9.823708e+09,1528981.875,1869379.000,...,2404.663818,337856608.0,310962.46875,3.109415e+09,2002417.000,338893600.0,308761.84375,3.135328e+09,1997065.250,337238208.0
1,2959.0,236281.0,3483336.0,3952939.0,1335.903687,4.858620e+08,164198.031250,4.268761e+09,1442636.500,1769694.125,...,2457.227051,328497920.0,310578.12500,3.072270e+09,1999905.875,335243104.0,306964.84375,3.140742e+09,1988548.625,333772448.0
2,10235.0,1396460.0,6005368.0,8788053.0,858.627563,1.046455e+09,102242.835938,1.199413e+10,1171874.375,1375504.250,...,2471.866455,332505152.0,316542.68750,3.094534e+09,2028123.250,341046080.0,308040.53125,3.193330e+09,1999627.625,336064192.0
3,105.0,2598.0,447860.0,452951.0,4313.818848,6.886670e+07,655873.375000,3.422819e+08,3259827.750,4567261.500,...,2376.960205,344903008.0,309467.56250,3.211684e+09,2016765.125,339384704.0,310340.50000,3.149223e+09,2014740.875,339198272.0
4,4360.0,504545.0,4665195.0,5669925.0,1300.441528,7.075592e+08,162284.218750,6.522889e+09,1496075.375,1819343.500,...,2404.033936,333137120.0,308907.03125,3.097117e+09,2005056.875,336754208.0,307344.87500,3.136502e+09,1997853.125,335776928.0


In [5]:
df.shape

(943475, 52)

In [8]:
from sklearn.decomposition import NMF
import numpy as np

def description_length_error(V, G, F):
    B = 64
    M = B * V.shape[1] * (V.shape[0] + F.shape[1])
    GF = G.dot(F)
    E = (V*np.ma.log(np.ma.divide(V, GF)) - V + GF).sum()
    return -(M + E)


def rolx_error(estimator, X, y):
    G = estimator.transform(X)
    F = estimator.components_
    return description_length_error(y, G, F)

In [32]:
V = df.iloc[:, 1:].values
model = NMF(n_components=31)
model.fit(V)
G = model.transform(V)
F = model.components_

B = 64
M = B * V.shape[1] * (V.shape[0] + F.shape[1])
GF = G.dot(F)
E = (V*np.ma.log(np.ma.divide(V,GF)) - V + GF).sum()
M+E

3280237805554.9614

In [33]:
%time rolx_error(model, V, V)

CPU times: user 12min 53s, sys: 3min 36s, total: 16min 29s
Wall time: 4min 36s


-3280237805554.9614

In [19]:
V = df.iloc[:, 1:].values

nmf = NMF(n_components=16, solver='mu', beta_loss='kullback-leibler', max_iter=1000)
F = nmf.fit_transform(V)
G = nmf.components_

In [39]:
pd.concat([df.iloc[:, :1].astype('int'), pd.DataFrame(F)], axis=1).to_csv("data/processed/role-nmf-v1.csv")

In [41]:
pd.DataFrame(G).to_csv("data/processed/role-nmf-G-v1.csv")

In [15]:
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

V = df.iloc[:, 1:].values


def error(estimator, X, y):
    return -estimator.reconstruction_err_

n_iter_search = 8
random_search = RandomizedSearchCV(
    NMF(solver='mu', beta_loss='kullback-leibler'), 
    param_distributions={"n_components": sp_randint(1, 64)},
    n_iter=n_iter_search,
    scoring=error
)

start = time()
random_search.fit(V, V)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))

/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/decomposition/nmf.py:1035: ConvergenceWarning: Maximum number of iteration 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/decomposition/nmf.py:1035: ConvergenceWarning: Maximum number of iteration 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/decomposition/nmf.py:1035: ConvergenceWarning: Maximum number of iteration 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/decomposition/nmf.py:1035: ConvergenceWarning: Maximum number of iteration 200 reached. Increase it to improve

RandomizedSearchCV took 10823.21 seconds for 8 candidates parameter settings.


In [17]:
random_search.cv_results_

/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/amiyaguchi/cs224w/wikipedia-retention/venv3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_scor

{'mean_fit_time': array([444.04070346, 414.07480009, 486.50003831, 308.77473815,
        378.06476402, 452.67154829, 512.01460648, 397.73086945]),
 'std_fit_time': array([14.82358931,  1.60803212, 29.53071736,  4.53529295, 29.0272755 ,
         3.02168036, 20.7969531 ,  4.29750798]),
 'mean_score_time': array([8.05854797e-05, 7.93139140e-05, 9.70363617e-05, 7.46250153e-05,
        8.47975413e-05, 7.18434652e-05, 9.56853231e-05, 7.62144725e-05]),
 'std_score_time': array([6.91824824e-06, 5.71652429e-06, 3.18667339e-05, 1.36267568e-06,
        2.05882262e-05, 4.05233662e-07, 2.29578921e-05, 1.12391596e-07]),
 'param_n_components': masked_array(data=[44, 27, 60, 21, 28, 44, 62, 44],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_components': 44},
  {'n_components': 27},
  {'n_components': 60},
  {'n_components': 21},
  {'n_components': 28},
  {'n_components': 44},
  {'n_components': 62},
  {'